In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU,Dense
from tensorflow.keras import callbacks
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import random
from tensorflow.keras.optimizers import Nadam
from keras.models import load_model
import matplotlib.pyplot as plt


2023-03-28 17:40:38.390164: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 17:40:38.525049: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-28 17:40:38.525074: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-28 17:40:38.554662: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-28 17:40:39.437250: W tensorflow/stream_executor/platform/de

In [3]:
import sys
sys.path.insert(0,'/home/rahmah/code/marvinliebisch/forecasting-electricity-prices')

In [4]:
from energy_price_predictions.ml_logic.data_import import *
from energy_price_predictions.ml_logic.preprocessing import *
from energy_price_predictions.ml_logic.full_pipeline import *
from energy_price_predictions.ml_logic.registry import *
from energy_price_predictions.ml_logic.model import *

Importing energy_dataset data from /home/rahmah/code/marvinliebisch/forecasting-electricity-prices/raw_data/energy_dataset.csv...
Importing weather_features data from /home/rahmah/code/marvinliebisch/forecasting-electricity-prices/raw_data/weather_features.csv...


2023-03-28 17:40:47.290579: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-28 17:40:47.290711: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-28 17:40:47.290767: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-28 17:40:47.290819: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-28 17:40:47.290870: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric_MinMaxScaler()',
                                                  MinMaxScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f7fcbb13fa0>),
                                                 ('categorical',
                                                  OneHotEncoder(),
                                                  ['valencia_weather_main',
                                                   'madrid_weather_main',
                                                   'bilbao_weather_main',
                                                   'barcelona_weather_main']),
                                                 ('month_sin',
                                                  FunctionTransformer(fun...
                                                  FunctionTransformer(func=<function cos_transformer.<locals>.<lam

In [6]:
df = import_merged_data()
y=df[['price_day_ahead']]
preprocessor  = run_pipeline(df)
X_preprocessed = pd.DataFrame(preprocessor.fit_transform(df))
X_preprocessed

Importing energy_dataset data from /home/rahmah/code/marvinliebisch/forecasting-electricity-prices/raw_data/energy_dataset.csv...
Importing weather_features data from /home/rahmah/code/marvinliebisch/forecasting-electricity-prices/raw_data/weather_features.csv...


ValueError: A given column is not a column of the dataframe

In [ ]:
def sequence_data(X, y,
                  n_observation_X, n_observation_y,
                  n_sequence_train,n_sequence_val,n_sequence_test,
                  val_cutoff,test_cutoff):


    sample_list_train = list(range(0, int(len(X)*val_cutoff-n_observation_y-n_observation_X)))
    sample_list_val = list(range(int(len(X)*val_cutoff),int(len(X)*test_cutoff)))
    sample_list_test= list(range(int(len(X)*test_cutoff),int(len(X)-n_observation_y-n_observation_X)))

    random.shuffle(sample_list_train)
    random.shuffle(sample_list_val)
    random.shuffle(sample_list_test)

    X_train=np.zeros((n_sequence_train, n_observation_X, X.shape[1]))
    X_val=np.zeros((n_sequence_val, n_observation_X, X.shape[1]))
    X_test=np.zeros((n_sequence_test, n_observation_X, X.shape[1]))

    y_train=np.zeros((n_sequence_train, n_observation_y, 1))
    y_val=np.zeros((n_sequence_val, n_observation_y, 1))
    y_test=np.zeros((n_sequence_test, n_observation_y, 1))


    def create_sequence(X_,y_,sample_list,n_sequence):
        index=0
        for i in sample_list[0:n_sequence]:
            X_[index] = X.iloc[i:i + n_observation_X].values
            y_[index]= y.iloc[i:i + n_observation_y].values

            index=index+1
        return X_, y_

    X_train, y_train = create_sequence(X_train,y_train,sample_list_train,n_sequence_train)
    X_val, y_val = create_sequence(X_val,y_val,sample_list_val,n_sequence_val)
    X_test, y_test = create_sequence(X_test,y_test,sample_list_test,n_sequence_test)

    return X_train, X_val, X_test, y_train, y_val,y_test


n_observation_X=24 * 7*4  # For example, a week of data for the sequence
n_observation_y=24 # We would like to forecast the 24 prices of the next day during the auction of today
n_sequence_train=200
n_sequence_val=100
n_sequence_test=100
val_cutoff=0.8
test_cutoff=0.9

X_train, X_val, X_test, y_train, y_val,y_test = sequence_data(X_preprocessed, y,
                  n_observation_X, n_observation_y,
                  n_sequence_train,n_sequence_val,n_sequence_test,
                  val_cutoff,test_cutoff)

In [ ]:
# Baseline model

y_flaten=np.reshape(y_train,(-1, 1))
y_flaten=pd.DataFrame(y_flaten)
y_true_baseline=y_flaten[24:]
y_pred_baseline= y_flaten.shift(24)[24:]
print(f"MAE: {round(mean_absolute_error(y_true_baseline, y_pred_baseline),2)} and the mean of day ahead price is : {round(y_true_baseline.mean()[0],2)}")

In [8]:
model = Sequential()
model.add(LSTM(units=16, input_shape=X_train.shape[1:], return_sequences=True))
model.add(LSTM(units=32, return_sequences=True))
model.add(LSTM(units=64, return_sequences=False))
model.add(Dense(24, activation='linear'))  # Output layer with 24 neurons (one for each hour)

# Compile and train the model
initial_learning=0.01 # Default value is 0.001
optimizer = Nadam(lr=initial_learning) #change the optimizer and right all the default value
model.compile(optimizer=optimizer, loss='mse',metrics=['mae'])
es = callbacks.EarlyStopping(patience=5, restore_best_weights=True)
model.fit(X_train, y_train, epochs=30, batch_size=16, callbacks=[es], validation_data=(X_val,y_val),verbose=1,shuffle=False)


model.save('model_lstm.h5')

In [ ]:
@mlflow_run
def train(X_train, y_train, X_val, y_val) -> float:
    """
    - Train on the preprocessed dataset (which should be ordered by date)
    - Store training results and model weights

    Return val_mae as float
    """

    # Train model using `model.py`
    # model = load_model()
    # if model is None:
    model = initialize_model(input_shape=X_train.shape[1:])
    model = compile_model(model)
    model, history = train_model(model, X_train, y_train,
                                 validation_data=(X_val, y_val))

    val_mae = np.min(history.history['val_mae'])

    params = dict(
        context="train",
        training_set_size=len(X_train),
    )

    save_results(params=params, metrics=dict(mae=val_mae))
    save_model(model=model)

    # The latest model should be moved to staging
    mlflow_transition_model(current_stage="None", new_stage="Staging")

    return val_mae

In [ ]:
@mlflow_run
def evaluate(X_new, y_new, stage: str = "Production") -> float:
    """
    Evaluate the performance of the latest production model on processed data
    Return mae as float
    """

    model = load_model(stage=stage)
    assert model is not None

    metrics_dict = evaluate_model(model=model, X=X_new, y=y_new)
    mae = metrics_dict["mae"]

    params = dict(
        context="evaluate", # Package behavior
    )

    save_results(params=params, metrics=metrics_dict)

    return mae


In [ ]:
df = import_merged_data()
X, y = run_pipeline(df)
X.shape

In [ ]:
X_train, X_val, X_test, y_train, y_val,y_test = split_train_test(X, y)
X_train

In [ ]:
train(X_train, X_val, y_train, y_val)